In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_excel('dataset_without_duplicates.xlsx')

In [3]:
df.columns

Index(['Отметка времени', 'Единая учетная запись СПбГУ (например, ST000000)',
       'Выберете интересующие Вас роли, в  IT [Бизнес и системный анализ]',
       'Выберете интересующие Вас роли, в  IT [Дизайн UX / UI и проектирование интерфейсов]',
       'Выберете интересующие Вас роли, в  IT [Анализ и инженерия данных]',
       'Выберете интересующие Вас роли, в  IT [Backend - разработка]',
       'Выберете интересующие Вас роли, в  IT [Frontend - разработка]',
       'Выберете интересующие Вас роли, в  IT [Тестирование и обеспечение качества]',
       'Выберете интересующие Вас роли, в  IT [Развертывание и внедрение]',
       'Выберете интересующие Вас роли, в  IT [Управление командой / проектом]',
       'Выберете интересные для вас направление проекта (в направлении проекта могут быть все роли, указанные выше) [Разработка веб-сервисов (сайты, сервисы)]',
       'Выберете интересные для вас направление проекта (в направлении проекта могут быть все роли, указанные выше) [Анализ текст

In [12]:
df['Year'] = df['Отметка времени'].dt.year
df['Season'] = np.where(df['Отметка времени'].dt.month > 7, 'Осень', 'Весна')

In [13]:
df

,Отметка времени,"Единая учетная запись СПбГУ (например, ST000000)","Выберете интересующие Вас роли, в IT [Бизнес и системный анализ]","Выберете интересующие Вас роли, в IT [Дизайн UX / UI и проектирование интерфейсов]","Выберете интересующие Вас роли, в IT [Анализ и инженерия данных]","Выберете интересующие Вас роли, в IT [Backend - разработка]","Выберете интересующие Вас роли, в IT [Frontend - разработка]","Выберете интересующие Вас роли, в IT [Тестирование и обеспечение качества]","Выберете интересующие Вас роли, в IT [Развертывание и внедрение]","Выберете интересующие Вас роли, в IT [Управление командой / проектом]",...,Оцените Ваш уровень мотивации [Устроиться на интересную для себя работу],"Хотите ли Вы общаться с другими студентами и выпускниками, заинтересованными в развитии в области IT на базе СПбГУ на интересующие Вас темы?","Что Вам поможет развиваться быстрее в том направлении деятельности, которое Вы выбрали?",Year,Season,score,hard_skills_score,soft_skills_score,analytics_skills_score,knowladges_skills_score
0,2023-10-07 16:02:10.676,ST110311,3,1,1.0,1,1,3,1,3,...,3,2,NaN,2023,Осень,10.1,1.2,4.6,3.3,1.0
1,2023-10-02 16:23:02.340,ST063118,1,3,1.0,5,3,3,3,3,...,4,1,"Опытные наставники, интересные задачи",2023,Осень,13.7,2.2,3.4,4.2,3.9
2,2023-09-15 19:00:59.596,ST114946,1,3,1.0,3,5,3,1,1,...,3,2,Чёткий план действий,2023,Осень,13.7,2.5,3.6,4.5,3.1
3,2023-09-11 14:35:19.227,ST112047,3,3,3.0,1,1,3,1,5,...,6,1,Прежний опыт в написанию проектов и любовь к р...,2023,Осень,10.8,1.0,5.0,3.7,1.1
4,2023-09-10 22:04:15.396,ST112471,1,1,1.0,1,1,3,1,5,...,1,1,Личный рост,2023,Осень,11.3,0.0,4.6,4.8,1.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,2019-08-26 11:39:44.782,ST061362,4,5,NaN,1,1,2,2,5,...,6,1,Время для развития навыков\nВозможность общать...,2019,Осень,11.4,2.6,2.4,3.5,2.9
345,2019-08-26 11:38:34.578,ST061177,1,5,NaN,4,4,1,2,2,...,3,1,NaN,2019,Осень,7.0,1.7,1.8,1.7,1.8
346,2019-08-26 11:37:09.651,ST054461,1,5,NaN,4,4,2,2,2,...,6,1,Чётко поставленные задачи,2019,Осень,11.0,1.8,2.2,4.0,3.0
347,2019-08-26 11:36:51.336,ST063110,1,2,NaN,4,4,4,2,1,...,6,1,"Считаю, что интересная, хорошо выстроенная задача",2019,Осень,9.1,2.0,1.6,2.7,2.8


In [14]:
def get_element_safely(lst, index=0, default=1):
    if 0 <= index < len(lst):
        return int(lst[index])
    else:
        return default

In [15]:
def evaluate_skills(skills_dict):
    if not skills_dict:
        return 0  
    
    score = list(skills_dict.values())
    total_score = sum(score)
    average_score = total_score / len(skills_dict)
    return average_score

In [25]:
def skills_score(df):
   pattern1 = r"\n?([a-zA-Zс+.#]+)"
   pattern2 = r"\d"
   data = df
   data['score'] = None
   data['hard_skills_score'] = None
   data['soft_skills_score'] = None
   data['analytics_skills_score'] = None
   data['knowladges_skills_score'] = None

   
   soft_skills_columns = [col for col in df.columns if 'SoftSkills' in col]
   analytics_skills_columns = [col for col in df.columns if 'Аналитические' in col]
   knowladges_skills_columns = [col for col in df.columns if 'знания в области' in col]      
   students = []
   for _, student_df in data.iterrows():
      student = dict()
      student['st'] = student_df['Единая учетная запись СПбГУ (например, ST000000)'] 
      student['year'] = student_df['Year']
      student['season'] =  student_df['Season']

      student['languages'] = {re.findall(pattern1, x)[0]: get_element_safely(re.findall(pattern2, x)) for x in str(student_df['Языки программирования (например: C++, Python, Java, etc)']).lower().split(sep='\n') if (pd.notna(x)) and (re.findall(pattern1, x) != [])} 
      student['languages_score'] = evaluate_skills(student['languages'])
    
      
      student['ci_cd'] = {re.findall(pattern1, x)[0]: get_element_safely(re.findall(pattern2, x)) for x in str(student_df['Инструменты CI / CD (например: Jenkins, Travis CI, Docker, etc)']).lower().split(sep='\n') if (pd.notna(x)) and (re.findall(pattern1, x) != [])} 
      student['ci_cd_score'] = evaluate_skills(student['ci_cd'])
          
      student['analysis'] = {re.findall(pattern1, x )[0]: get_element_safely(re.findall(pattern2, x)) for x in str(student_df['Инструменты для проектирования, бизнес и системного анализа (например: BPMN, UML, Archimate, ARIS etc)']).lower().split(sep='\n') if (pd.notna(x)) and (re.findall(pattern1, x) != [])} 
      student['analysis_score'] = evaluate_skills(student['analysis'])
     
      student['test'] = {re.findall(pattern1, x)[0]: get_element_safely(re.findall(pattern2, x)) for x in str(student_df['Инструменты для тестирования  (например: GTests, Selenium, Gatling, pytest etc)']).lower().split(sep='\n') if (pd.notna(x)) and (re.findall(pattern1, x) != [])} 
      student['test_score'] = evaluate_skills(student['test'])
    
      student['frameworks'] = {re.findall(pattern1, x )[0]: get_element_safely(re.findall(pattern2, x)) for x in str(student_df['Библиотеки и фреймворки для различных направлений использования (например: Qt, Numpy, Weka, Angular, PyTorch, etc)']).lower().split(sep='\n') if (pd.notna(x)) and (re.findall(pattern1, x) != [])} 
      student['frameworks_score'] = evaluate_skills(student['frameworks'])
    
      student['dev'] = {re.findall(pattern1, x)[0]: get_element_safely(re.findall(pattern2, x)) for x in str(student_df['Инструменты для разработки ПО - например: VS Code, PyCharm, Git, etc)']).lower().split(sep='\n') if (pd.notna(x)) and (re.findall(pattern1, x) != [])} 
      student['dev_score'] = evaluate_skills(student['dev'])
      
      student['hard_skills_score'] = np.round(student['dev_score']*1/6+student['frameworks_score']*1/6+student['languages_score']*1/6+student['ci_cd_score']*1/6+student['test_score']*1/6+student['analysis_score']*1/6, 1)    
      student['soft_skills_score'] = np.round(student_df[soft_skills_columns].fillna(0).mean(), 1)
      student['analytics_skills_score'] = np.round(student_df[analytics_skills_columns].fillna(0).mean(), 1)
      student['knowladges_skills_score'] = np.round(student_df[knowladges_skills_columns].fillna(0).mean(), 1)
      
      student['score'] = np.round(student['hard_skills_score'] + student['soft_skills_score']+student['analytics_skills_score']+student['knowladges_skills_score'], 1)

      students.append(student)
      
      data.loc[(data['Единая учетная запись СПбГУ (например, ST000000)'] == student['st']) & (data['Year'] == int(student['year'])) & (data['Season'] == student['season']), 'score'] = student['score']
      data.loc[(data['Единая учетная запись СПбГУ (например, ST000000)'] == student['st']) & (data['Year'] == int(student['year'])) & (data['Season'] == student['season']), 'hard_skills_score'] = student['hard_skills_score']
      data.loc[(data['Единая учетная запись СПбГУ (например, ST000000)'] == student['st']) & (data['Year'] == int(student['year'])) & (data['Season'] == student['season']), 'soft_skills_score'] = student['soft_skills_score']
      data.loc[(data['Единая учетная запись СПбГУ (например, ST000000)'] == student['st']) & (data['Year'] == int(student['year'])) & (data['Season'] == student['season']), 'analytics_skills_score'] = student['analytics_skills_score']
      data.loc[(data['Единая учетная запись СПбГУ (например, ST000000)'] == student['st']) & (data['Year'] == int(student['year'])) & (data['Season'] == student['season']), 'knowladges_skills_score'] = student['knowladges_skills_score']
      

   return data, students

In [28]:
df, students_with_score = skills_score(df)
students_with_score

[{'st': 'ST110311',
  'year': 2023,
  'season': 'Осень',
  'languages': {'python': 2},
  'languages_score': 2.0,
  'ci_cd': {'nan': 1},
  'ci_cd_score': 1.0,
  'analysis': {'nan': 1},
  'analysis_score': 1.0,
  'test': {'nan': 1},
  'test_score': 1.0,
  'frameworks': {'numpy': 1},
  'frameworks_score': 1.0,
  'dev': {'nan': 1},
  'dev_score': 1.0,
  'hard_skills_score': 1.2,
  'soft_skills_score': 4.6,
  'analytics_skills_score': 3.3,
  'knowladges_skills_score': 1.0,
  'score': 10.1},
 {'st': 'ST063118',
  'year': 2023,
  'season': 'Осень',
  'languages': {'c#': 4, 'c': 3, 'c++': 2, 'python': 3},
  'languages_score': 3.0,
  'ci_cd': {'docker': 4},
  'ci_cd_score': 4.0,
  'analysis': {},
  'analysis_score': 0,
  'test': {},
  'test_score': 0,
  'frameworks': {'asp.net': 4, 'blazor': 4, 'numpy': 3, 'qt': 2},
  'frameworks_score': 3.25,
  'dev': {'rider': 4, 'git': 4, 'pycharm': 3, 'svn': 2},
  'dev_score': 3.25,
  'hard_skills_score': 2.2,
  'soft_skills_score': 3.4,
  'analytics_skills

In [29]:
df

,Отметка времени,"Единая учетная запись СПбГУ (например, ST000000)","Выберете интересующие Вас роли, в IT [Бизнес и системный анализ]","Выберете интересующие Вас роли, в IT [Дизайн UX / UI и проектирование интерфейсов]","Выберете интересующие Вас роли, в IT [Анализ и инженерия данных]","Выберете интересующие Вас роли, в IT [Backend - разработка]","Выберете интересующие Вас роли, в IT [Frontend - разработка]","Выберете интересующие Вас роли, в IT [Тестирование и обеспечение качества]","Выберете интересующие Вас роли, в IT [Развертывание и внедрение]","Выберете интересующие Вас роли, в IT [Управление командой / проектом]",...,Оцените Ваш уровень мотивации [Устроиться на интересную для себя работу],"Хотите ли Вы общаться с другими студентами и выпускниками, заинтересованными в развитии в области IT на базе СПбГУ на интересующие Вас темы?","Что Вам поможет развиваться быстрее в том направлении деятельности, которое Вы выбрали?",Year,Season,score,hard_skills_score,soft_skills_score,analytics_skills_score,knowladges_skills_score
0,2023-10-07 16:02:10.676,ST110311,3,1,1.0,1,1,3,1,3,...,3,2,NaN,2023,Осень,10.1,1.2,4.6,3.3,1.0
1,2023-10-02 16:23:02.340,ST063118,1,3,1.0,5,3,3,3,3,...,4,1,"Опытные наставники, интересные задачи",2023,Осень,13.7,2.2,3.4,4.2,3.9
2,2023-09-15 19:00:59.596,ST114946,1,3,1.0,3,5,3,1,1,...,3,2,Чёткий план действий,2023,Осень,13.7,2.5,3.6,4.5,3.1
3,2023-09-11 14:35:19.227,ST112047,3,3,3.0,1,1,3,1,5,...,6,1,Прежний опыт в написанию проектов и любовь к р...,2023,Осень,10.8,1.0,5.0,3.7,1.1
4,2023-09-10 22:04:15.396,ST112471,1,1,1.0,1,1,3,1,5,...,1,1,Личный рост,2023,Осень,11.3,0.0,4.6,4.8,1.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,2019-08-26 11:39:44.782,ST061362,4,5,NaN,1,1,2,2,5,...,6,1,Время для развития навыков\nВозможность общать...,2019,Осень,11.4,2.6,2.4,3.5,2.9
345,2019-08-26 11:38:34.578,ST061177,1,5,NaN,4,4,1,2,2,...,3,1,NaN,2019,Осень,7.0,1.7,1.8,1.7,1.8
346,2019-08-26 11:37:09.651,ST054461,1,5,NaN,4,4,2,2,2,...,6,1,Чётко поставленные задачи,2019,Осень,11.0,1.8,2.2,4.0,3.0
347,2019-08-26 11:36:51.336,ST063110,1,2,NaN,4,4,4,2,1,...,6,1,"Считаю, что интересная, хорошо выстроенная задача",2019,Осень,9.1,2.0,1.6,2.7,2.8


In [18]:
students_with_score

[{'st': 'ST110311',
  'year': 2023,
  'season': 'Осень',
  'languages': {'python': 2},
  'languages_score': 2.0,
  'ci_cd': {'nan': 1},
  'ci_cd_score': 1.0,
  'analysis': {'nan': 1},
  'analysis_score': 1.0,
  'test': {'nan': 1},
  'test_score': 1.0,
  'frameworks': {'numpy': 1},
  'frameworks_score': 1.0,
  'dev': {'nan': 1},
  'dev_score': 1.0,
  'hard_skills_score': 1.2,
  'soft_skills_score': 4.6,
  'analytics_skills_score': 3.3,
  'knowladges_skills_score': 1.0,
  'score': 10.1},
 {'st': 'ST063118',
  'year': 2023,
  'season': 'Осень',
  'languages': {'c#': 4, 'c': 3, 'c++': 2, 'python': 3},
  'languages_score': 3.0,
  'ci_cd': {'docker': 4},
  'ci_cd_score': 4.0,
  'analysis': {},
  'analysis_score': 0,
  'test': {},
  'test_score': 0,
  'frameworks': {'asp.net': 4, 'blazor': 4, 'numpy': 3, 'qt': 2},
  'frameworks_score': 3.25,
  'dev': {'rider': 4, 'git': 4, 'pycharm': 3, 'svn': 2},
  'dev_score': 3.25,
  'hard_skills_score': 2.2,
  'soft_skills_score': 3.4,
  'analytics_skills

In [30]:
df.to_excel('data_with_score.xlsx', index=False)